In [ ]:
import keras_tuner as kt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

def build_model(hp):
    model = Sequential()
    
    # Conv Layer 1
    model.add(Conv2D(
        filters=hp.Choice('conv1_filters', [16, 32, 64]),  
        kernel_size=hp.Choice('conv1_kernel', [3, 5]),  
        activation='relu',
        input_shape=(48, 48, 1)
    ))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    
    # Conv Layer 2
    model.add(Conv2D(
        filters=hp.Choice('conv2_filters', [32, 64, 128]),
        kernel_size=hp.Choice('conv2_kernel', [3, 5]),
        activation='relu'
    ))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    
    # Conv Layer 3
    model.add(Conv2D(
        filters=hp.Choice('conv3_filters', [64, 128, 256]),
        kernel_size=3,
        activation='relu'
    ))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    
    # Dense Layers
    model.add(Flatten())
    model.add(Dense(
        units=hp.Choice('dense_units', [64, 128, 256]),
        activation='relu'
    ))
    model.add(Dropout(hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1)))
    
    # Output Layer
    model.add(Dense(7, activation='softmax'))
    
    # Compile Model
    model.compile(
        optimizer=hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop']),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Create a tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=20,
    factor=3,
    directory='hyperparameter_tuning',
    project_name='emotion_recognition'
)

# Load Data
train_dataset = tf.keras.utils.image_dataset_from_directory(
    'dataset/train',
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(48, 48),
    batch_size=32,
    color_mode="grayscale"
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    'dataset/train',
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(48, 48),
    batch_size=32,
    color_mode="grayscale"
)

# Run the search
tuner.search(train_dataset, validation_data=validation_dataset, epochs=20)

# Get the best model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)

# Train the best model
history = model.fit(train_dataset, validation_data=validation_dataset, epochs=50)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Paths to the dataset
train_dir = '/home/laptop-kl-11/personal_project/face_expression_classification_guvi/dataset/train'
test_dir = '/home/laptop-kl-11/personal_project/face_expression_classification_guvi/dataset/test'

# Constants
IMG_HEIGHT, IMG_WIDTH = 48, 48
BATCH_SIZE = 32
EPOCHS = 200
AUTOTUNE = tf.data.AUTOTUNE

# Load Train Dataset and Split into Training and Validation
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split=0.25,  # 20% of training data for validation
    subset="training",
    seed=123,
    color_mode="grayscale",
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split=0.25,  # 20% of training data for validation
    subset="validation",
    seed=123,
    color_mode="grayscale",
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

# Load Test Dataset
test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    color_mode="grayscale",
    batch_size=BATCH_SIZE
)

# Data Augmentation for Training
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2)
])

# Apply data augmentation to the training dataset
def augment_data(image, label):
    image = data_augmentation(image)
    return image, label

train_dataset = train_dataset.map(augment_data, num_parallel_calls=AUTOTUNE)


train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(7, activation='softmax')  
])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',  
    metrics=['accuracy']
)


callbacks = [
    EarlyStopping(
        monitor='val_loss',  
        patience=25,         
        restore_best_weights=True  
    ),
    ModelCheckpoint(
        filepath='/kaggle/working/emotion_model_tf_data.keras', 
        monitor='val_loss',
        save_best_only=True
    )
]

#training model
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=EPOCHS,
    callbacks = callbacks
)




Found 28709 files belonging to 7 classes.
Using 21532 files for training.
Found 28709 files belonging to 7 classes.
Using 7177 files for validation.
Found 7178 files belonging to 7 classes.
Epoch 1/200
673/673 ━━━━━━━━━━━━━━━━━━━━ 21s 24ms/step - accuracy: 0.2363 - loss: 2.4001 - val_accuracy: 0.2476 - val_loss: 1.8162
Epoch 2/200
673/673 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.2499 - loss: 1.8077 - val_accuracy: 0.2735 - val_loss: 1.7607
Epoch 3/200
673/673 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.2713 - loss: 1.7617 - val_accuracy: 0.2954 - val_loss: 1.7072
Epoch 4/200
673/673 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.2971 - loss: 1.7245 - val_accuracy: 0.3488 - val_loss: 1.6622
Epoch 5/200
673/673 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.3399 - loss: 1.6542 - val_accuracy: 0.3915 - val_loss: 1.5642
Epoch 6/200
673/673 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.3735 - loss: 1.5948 - val_accuracy: 0.4074 - val_loss: 1.5409
Epoch 7/200
673/673 ━━━━━━━━━━━━━━

In [13]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

#evaluation with test dataset
loss, accuracy = model.evaluate(test_dataset, verbose=0)
print("Loss: %2.6f" % loss)

# Get predictions and ground truth
y_gt = []
y_pred_top1 = []
y_pred_probs = []

# Iterate over the test dataset
for images, labels in test_dataset:
    # Ground truth
    y_gt.extend(labels.numpy())
    
    # Predictions
    probs = model.predict(images, verbose=0)  # Probabilities for each class
    y_pred_top1.extend(np.argmax(probs, axis=1))  # Top-1 predictions
    y_pred_probs.extend(probs)

y_gt = np.array(y_gt)
y_pred_top1 = np.array(y_pred_top1)
y_pred_probs = np.array(y_pred_probs)

# Calculate Top-1 Accuracy
acc1 = np.mean(y_pred_top1 == y_gt) * 100
print("Top 1 Accuracy: %2.6f %%" % acc1)

# Calculate Top-2 Accuracy
top2_correct = 0
for i in range(len(y_gt)):
    top2_preds = np.argsort(y_pred_probs[i])[-2:]  # Top-2 predictions
    if y_gt[i] in top2_preds:
        top2_correct += 1
acc2 = (top2_correct / len(y_gt)) * 100
print("Top 2 Accuracy: %2.6f %%" % acc2)

# Calculate Precision, Recall, and F1 Score
precision = precision_score(y_gt, y_pred_top1, average='micro')
recall = recall_score(y_gt, y_pred_top1, average='micro')
f1 = f1_score(y_gt, y_pred_top1, average='micro')

print("Precision: %2.6f" % precision)
print("Recall: %2.6f" % recall)
print("F1 Score: %2.6f" % f1)

# Confusion Matrix
conf_matrix = confusion_matrix(y_gt, y_pred_top1)
print("Confusion Matrix:\n", conf_matrix, '\n')


Loss: 1.122096
Top 1 Accuracy: 58.498189 %
Top 2 Accuracy: 76.567289 %
Precision: 0.584982
Recall: 0.584982
F1 Score: 0.584982
Confusion Matrix:
 [[ 485    0   72   88  111  189   13]
 [  48    9   25    6    5   16    2]
 [ 160    1  308   79  125  280   71]
 [  44    0   34 1555   80   48   13]
 [  81    0   53  139  682  262   16]
 [ 139    0   93  121  249  631   14]
 [  31    0  115   73   54   29  529]] 



2024-12-25 17:37:21.913996: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
